# Introduction to NeuralHydrology

**Before we start**

- This tutorial is rendered from a Jupyter notebook that is hosted on GitHub. If you want to run the code yourself, you can find the notebook and configuration files [here](https://github.com/neuralhydrology/neuralhydrology/tree/master/examples/01-Introduction).
- To be able to run this notebook locally, you need to download the publicly available CAMELS US rainfall-runoff dataset. See the [Data Prerequisites Tutorial](data-prerequisites.nblink) for a detailed description on where to download the data and how to structure your local dataset folder. You will also need to follow the [installation instructions](https://neuralhydrology.readthedocs.io/en/latest/usage/quickstart.html#installation) (the easiest option if you don't plan to implement your own models/datasets is `pip install neuralhydrology`; for other options refer to the installation instructions).

The Python package NeuralHydrology was was developed with a strong focus on research. The main application area is hydrology, however, in principle the code can be used with any data. To allow fast iteration of research ideas, we tried to develop the package as modular as possible so that new models, new data sets, new loss functions, new regularizations, new metrics etc. can be integrated with minor effort.

There are two different ways to use this package:

1. From the terminal, making use of some high-level entry points (such as `nh-run` and `nh-run-scheduler`)
2. From any other Python file or Jupyter Notebook, using NeuralHydrology's API

In this tutorial, we will give a very short overview of the two different modes.

Both approaches require a **configuration file**. These are `.yml` files which define the entire run configuration (such as data set, basins, data periods, model specifications, etc.). A full list of config arguments is listed in the [documentation](https://neuralhydrology.readthedocs.io/en/latest/usage/config.html) and we highly recommend to check this page and read the documentation carefully. There is a lot that you can do with this Python package and we can't cover everything in tutorials.

For every run that you start, a new folder will be created. This folder is used to store the model and optimizer checkpoints, train data means/stds (needed for scaling during inference), tensorboard log file (can be used to monitor and compare training runs visually), validation results (optionally) and training progress figures (optionally, e.g., model predictions and observations for _n_ random basins). During inference, the evaluation results will also be stored in this directory (e.g., test period results).


### TensorBoard logging
By default, the training progress is logged in TensorBoard files (add `log_tensorboard: False` to the config to disable TensorBoard logging). If you installed a Python environment from one of our environment files, you have TensorBoard already installed. If not, you can install TensorBoard with:

```
pip install tensorboard
``` 

To start the TensorBoard dashboard, run:

```
tensorboard --logdir /path/to/run-dir
```

You can also visualize multiple runs at once if you point the `--logdir` to the parent directory (useful for model intercomparison)

### File logging
In addition to TensorBoard, you will always find a file called `output.log` in the run directory. This file is a dump of the console output you see during training and evaluation.


## Using NeuralHydrology from the Terminal

### nh-run


Given a run configuration file, you can use the bash command `nh-run` to train/evaluate a model. To train a model, use


```bash
nh-run train --config-file path/to/config.yml
```

to evaluate the model after training, use

```bash
nh-run evaluate --run-dir path/to/run-directory
```

### nh-run-scheduler

If you want to train/evaluate multiple models on different GPUs, you can use the `nh-run-scheduler`. This tool automatically distributes runs across GPUs and starts a new one, whenever one run finishes.

Calling `nh-run-scheduler` in `train` mode will train one model for each `.yml` file in a directory (or its sub-directories).

```bash
nh-run-scheduler train --directory /path/to/config-dir --runs-per-gpu 2 --gpu_ids 0 1 2 3 
```
Use `-runs-per-gpu` to define the number of models that are simultaneously trained on a _single_ GPU (2 in this case) and `--gpu-ids` to define which GPUs will be used (numbers are ids according to nvidia-smi). In this example, 8 models will train simultaneously on 4 different GPUs.

Calling `nh-run-scheduler` in `evaluate` mode will evaluate all models in all run directories in a given root directory.

```bash
nh-run-scheduler evaluate --directory /path/to/parent-run-dir/ --runs-per-gpu 2 --gpu_ids 0 1 2 3 
```

## API usage

Besides the command line tools, you can also use the NeuralHydrology package just like any other Python package by importing its modules, classes, or functions.

This can be helpful for exploratory studies with trained models, but also if you want to use some of the functions or classes within a different codebase. 

Look at the [API Documentation](https://neuralhydrology.readthedocs.io/en/latest/api/neuralhydrology.html) for a full list of functions/classes you could use.

The following example shows how to train and evaluate a model via the API.

In [1]:
%load_ext autoreload
%autoreload 2

import pickle
from pathlib import Path
import yaml

import torch
import matplotlib.pyplot as plt
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run, continue_run

import os
# os.environ['LD_LIBRARY_PATH'] = '/home/ame805/neuralhydrology_fork/venv-nh/lib/python3.8/site-packages/nvidia/cudnn/lib'
# os.environ['LD_LIBRARY_PATH'] = '../.././venv-nh/lib/python3.8/site-packages/nvidia/cudnn/lib'
print(os.environ['LD_LIBRARY_PATH'])

from datetime import datetime

# Get the current date and format it as 'ddmm' (e.g., '0711' for November 7th)
current_date = datetime.now().strftime('%Y-%m-%d')
# current_date = '2024-03-04'

/home/ame805/neuralhydrology_fork/venv-nh/lib/python3.8/site-packages/nvidia/cudnn/lib


In [2]:
# Print torch version
print("Torch Version: ", torch.__version__)
# Print cuda version
print("Cuda Version: ", torch.version.cuda)
# Print cudaNN version
print("CudaNN Version: ", torch.backends.cudnn.version())

Torch Version:  2.1.0+cu121
Cuda Version:  12.1
CudaNN Version:  8902


### Train a model for a single config file

**Note**

- The config file assumes that the CAMELS US dataset is stored under `data/CAMELS_US` (relative to the main directory of this repository) or a symbolic link exists at this location. Make sure that this folder contains the required subdirectories `basin_mean_forcing`, `usgs_streamflow` and `camels_attributes_v2.0`. If your data is stored at a different location and you can't or don't want to create a symbolic link, you will need to change the `data_dir` argument in the `1_basin.yml` config file that is located in the same directory as this notebook.
- By default, the config (`1_basin.yml`) assumes that you have a CUDA-capable NVIDIA GPU (see config argument `device`). In case you don't have any or you have one but want to train on the CPU, you can either change the config argument to `device: cpu` or pass `gpu=-1` to the `start_run()` function.

In [3]:
# # os.listdir('../../../../../gladwell/hydrology/SUMMA/summa-ml-models/CAMELS_spat_NH/CAMELS_spat_CAN/CAN_01AD002.csv')
# list_dir = os.listdir('../../../../../gladwell/hydrology/SUMMA/summa-ml-models/CAMELS_spat_NH/CAMELS_spat_CAN')
# print(len(list_dir), 'CAN_01AD002.csv' in list_dir)

In [4]:
basin_file = "1_basin.yml"
# basin_file = "10_basin_camels_spat.yml"
# basin_file = "56_basin_camels_spat.yml"
# basin_file = "426_basin_camels_spat.yml"

# Load basin_file = "1_basin.yml" as a dictionary
with open(basin_file, "r") as f:
    run_config = yaml.safe_load(f)

# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
    start_run(config_file=Path(basin_file))

# fall back to CPU-only mode
else:
    start_run(config_file=Path(basin_file), gpu=-1)

# continue_run(run_dir=Path('runs/426_camels_spat_USA_2024-03-05_093239'))

2024-07-15 15:56:14,938: Logging to /home/ame805/neuralhydrology_fork/examples/01-Introduction/runs/test_run_1507_155614/output.log initialized.
2024-07-15 15:56:14,939: ### Folder structure created at /home/ame805/neuralhydrology_fork/examples/01-Introduction/runs/test_run_1507_155614
2024-07-15 15:56:14,940: ### Run configurations for test_run
2024-07-15 15:56:14,940: experiment_name: test_run
2024-07-15 15:56:14,941: train_basin_file: 1_basinB.txt
2024-07-15 15:56:14,941: validation_basin_file: 1_basinB.txt
2024-07-15 15:56:14,942: test_basin_file: 1_basinB.txt
2024-07-15 15:56:14,942: train_start_date: 1980-10-01 00:00:00
2024-07-15 15:56:14,943: train_end_date: 2000-09-30 00:00:00
2024-07-15 15:56:14,944: validation_start_date: 2000-10-01 00:00:00
2024-07-15 15:56:14,945: validation_end_date: 2010-09-30 00:00:00
2024-07-15 15:56:14,946: test_start_date: 2000-10-01 00:00:00
2024-07-15 15:56:14,947: test_end_date: 2010-09-30 00:00:00
2024-07-15 15:56:14,947: device: cuda:0
2024-07-1

### Evaluate run on test set
The run directory that needs to be specified for evaluation is printed in the output log above. Since the folder name is created dynamically (including the date and time of the start of the run) you will need to change the `run_dir` argument according to your local directory name. By default, it will use the same device as during the training process.

In [7]:
# # Select the last folder in the 'runs' directory that matches the current date
# run_files = sorted(os.listdir("runs"))

# # Filter filenames that match the current date
# exp_name = run_config["experiment_name"]

# display(exp_name, current_date)
# filtered_run_files = sorted([filename for filename in run_files if filename.replace(f'{exp_name}_', '').split('_')[0] == current_date])

# display(filtered_run_files)

# # Ensure there are matching filenames for the current date
# if filtered_run_files:
#     # Sort the filtered filenames by timestamp in descending order
#     # The latest filename for the current date is the last in the sorted list
#     run_file = filtered_run_files[-1]

run_file = 'test_run_1507_155614'

run_file

'test_run_1507_155614'

In [8]:
run_dir = Path(f"runs/{run_file}")
eval_run(run_dir=run_dir, period="test")

2024-07-15 15:59:53,463: Using the model weights from runs/test_run_1507_155614/model_epoch050.pt
# Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

# Evaluation: 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]
2024-07-15 15:59:53,956: Stored metrics at runs/test_run_1507_155614/test/model_epoch050/test_metrics_seed268159.csv
2024-07-15 15:59:53,958: Stored results at runs/test_run_1507_155614/test/model_epoch050/test_results.p


### Load and inspect model predictions
Next, we load the results file and compare the model predictions with observations. The results file is always a pickled dictionary with one key per basin (even for a single basin). The next-lower dictionary level is the temporal resolution of the predictions. In this case, we trained a model only on daily data ('1D'). Within the temporal resolution, the next-lower dictionary level are `xr`(an xarray Dataset that contains observations and predictions), as well as one key for each metric that was specified in the config file.

In [9]:
epoch_str = str(run_config["epochs"]).rjust(3, "0")

with open(run_dir / "test" / f'{"model_epoch" + epoch_str}' / "test_results.p", "rb") as fp:
    results = pickle.load(fp)
    
results.keys()

dict_keys(['06431500'])

The data variables in the xarray Dataset are named according to the name of the target variables, with suffix `_obs` for the observations and suffix `_sim` for the simulations.

In [10]:
def find_id_by_key(key, dictionary):
    id = 0
    for k, v in dictionary.items():
        if k == key:
            return id
        id += 1
    return None

In [11]:
basin_txt = run_config["train_basin_file"]

with open(basin_txt, "rb") as fp:
    hru_ids_bytes = fp.read()

hru_ids = hru_ids_bytes.decode("utf-8").splitlines()

HRU2PLOT = 0
# HRU2PLOT = find_id_by_key('CAN_08KH001', results)
# HRU2PLOT = find_id_by_key('USA_01013500', results)
# HRU2PLOT = find_id_by_key('USA_06431500', results)
hru_id = hru_ids[HRU2PLOT]

results[hru_id]['1D']['xr']

<xarray.Dataset>
Dimensions:         (date: 3652, time_step: 1)
Coordinates:
  * date            (date) datetime64[ns] 2000-10-01 2000-10-02 ... 2010-09-30
  * time_step       (time_step) int64 0
Data variables:
    QObs(mm/d)_obs  (date, time_step) float32 0.4181 0.441 ... 0.4525 0.4525
    QObs(mm/d)_sim  (date, time_step) float32 0.3432 0.3395 ... 0.4285 0.438

Let's plot the model predictions vs. the observations

In [12]:
target_var = run_config["target_variables"][0]
target_obs = f'{target_var}_obs'
target_sim = f'{target_var}_sim'


qobs = results[hru_id]['1D']['xr'][target_obs]
qsim = results[hru_id]['1D']['xr'][target_sim]

fig, ax = plt.subplots(figsize=(16,6))
ax.plot(qobs['date'], qobs, linewidth=3, label='Observed - NH')
ax.plot(qsim['date'], qsim, '--', label='Predicted - NH')
ax.set_ylabel("Discharge (mm/d)")
plt.legend()
ax.set_title(f"NeuralHydrology - Test period - {hru_id} - NSE {results[hru_id]['1D']['NSE']:.3f}")

Text(0.5, 1.0, 'NeuralHydrology - Test period - 06431500 - NSE 0.289')

Next, we are going to compute all metrics that are implemented in the NeuralHydrology package. You will find additional hydrological signatures implemented in `neuralhydrology.evaluation.signatures`.

In [ ]:
values = metrics.calculate_all_metrics(qobs.isel(time_step=-1), qsim.isel(time_step=-1))
for key, val in values.items():
    print(f"{key}: {val:.3f}")

NSE: 0.842
MSE: 0.998
RMSE: 0.999
KGE: 0.787
Alpha-NSE: 0.831
Beta-KGE: 0.891
Beta-NSE: -0.103
Pearson-r: 0.929
FHV: -14.555
FMS: -17.064
FLV: -14.150
Peak-Timing: 0.111
Peak-MAPE: 35.818
